# Assignment - APRAU Group 1

**Student - Student number**  
Ladislav Gardian - 1240524  
Lars van der Griend - 1240271  
Peter Likavec - 1240523  

ToDo:  
- Encode target variable (use OneHotEncoder) since there is no order in the target variable and regression models treat the encoded values as different classes and not as ordered values. Therefore, also maybe use factorize instead of dummies but need to figure that out
- further data preprocessing (dummys or factorize)

In [29]:
# Importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

For every group, three datasets where available for creating a model to predict the Vegetaton Type. This classification problem will be solved by applying Machine Learning models. To train and test such Machine Learning models, all three datasets are loaded and merged into one big dataset which will be used in the rest of the Notebook.

In [30]:
# Load data from the three available datasets for Group 1
df_1 = pd.read_csv("Data_Class_1.csv")
df_2 = pd.read_csv("Data_Class_2.csv")
df_3 = pd.read_csv("Data_Class_3.csv")

In [31]:
# Merging the DataFrames 
df = pd.concat([df_1, df_2, df_3], ignore_index=True)

In [ ]:
df.head()

### Exploratory Analysis

To get a better understanding of the data, an exploratory analysis of the dataset will be done. This exploratory analysis can give prior insights on the data, can help understand the data structure and dimensions better, and much more. The subsections below will guide through this exploratory analysis in steps.  

**Descriptive Statistics and Data Types**  
Using built-in Python functions, the descriptive statistics of all numerical columns in the dataset can be summarized, see the Table below. The *count* for every column equals 5184, indicating that there are no missing values. Furthermore, the data type per column is shown. Excluding the target variable *Vegetation_Type*, there are two more categorical columns; *Soil_Type* and *Wilderness_Area*. The other columns all contain numerical values. Moreover, the number of unique values per column is shown.

In [ ]:
df.describe()

In [ ]:
# Check data types and missing values
data_info = pd.DataFrame({
    'Data Type': df.dtypes,
    'Missing Values': df.isnull().sum(),
    'Unique Values': df.nunique()
})

data_info

**Univariate Analysis**  
Now the distribution of the data in individual columns (i.e., features) can be investigated. Understanding the distributions of the features ir=s relevant for later data analysis. Based on the distribution of the features, different methods should be applied. First, the distribution of the numerical features will be checked by creating histograms. The density curves are added to help understand the distributions. As can be seen, the features all have different distributions. Some Uniform, Gamma, Normal and other distributions can be recognized.

In [ ]:
# Set the style for the visualizations
sns.set(style="whitegrid")

# List of numerical columns to plot
numerical_columns = [
    'Altitude', 'Slope_Orientation', 'Slope', 
    'Horizontal_Distance_To_Water', 'Vertical_Distance_To_Water', 
    'Horizontal_Distance_To_Roadways', 'Shadow_Index_9h', 
    'Shadow_Index_12h', 'Shadow_Index_15h', 
    'Horizontal_Distance_To_Fire_Points', 'Canopy_Density', 
    'Rainfall_Summer', 'Rainfall_Winter', 'Wind_Exposure_Level'
]

# Set up the subplots, adjusting number of rows and columns to fit all features
num_plots = len(numerical_columns)
cols = 3
rows = num_plots // cols + (num_plots % cols > 0)

fig, axes = plt.subplots(rows, cols, figsize=(18, 12))
fig.suptitle('Distribution of Numerical Variables', fontsize=16)

# Plot histograms for each numerical feature
for i, col in enumerate(numerical_columns):
    row = i // cols
    col_idx = i % cols
    sns.histplot(df[col], kde=True, bins=20, ax=axes[row, col_idx])
    axes[row, col_idx].set_title(f'{col} Distribution')

# Hide any unused subplots
for i in range(num_plots, rows * cols):
    fig.delaxes(axes.flat[i])

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

Now, using barcharts the distribution of the non-numerical features (excluding the target variable) can be visualized. It can be seen that for the *Soil_Type* some values occur very often, and some occur only a few times. The *Wilderness_Area* distribution is a bit skewed as well.

In [ ]:
# Plot bar plots for categorical variables
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Distribution of Categorical Variables')

sns.countplot(x='Soil_Type', data=df, ax=axes[0], palette='Set2')
axes[0].tick_params(axis='x', rotation=45)
axes[0].set_title('Soil Type Distribution')

sns.countplot(x='Wilderness_Area', data=df, ax=axes[1], palette='Set2')
axes[1].set_title('Wilderness Area Distribution')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

**Bivariate Analysis**  
Now we can investigate the relationship between the individual features and the Vegetation Type. Since the problem of predicting the *Vegetation_Type* is a classification problem, to identify patterns between the numerical variables and target variable, violinplots are used. Violinplots give the distribution of the numerical value, given the target variable *Vegetation_Type*.

In [ ]:
# Set up the subplots, adjusting number of rows and columns to fit all features
num_plots = len(numerical_columns)
cols = 3  # Number of columns in the grid
rows = num_plots // cols + (num_plots % cols > 0)  # Calculate required rows

# Create subplots
fig, axes = plt.subplots(rows, cols, figsize=(18, 12))
fig.suptitle('Scatter Plots: Vegetation_Type vs Numerical Variables', fontsize=16)

# Plot violoinplots for each numerical feature against the target variable 'Vegetation_Type'
for i, col in enumerate(numerical_columns):
    row = i // cols
    col_idx = i % cols
    sns.violinplot(x=df['Vegetation_Type'], y=df[col], ax=axes[row, col_idx], palette='Set2')  # Vegetation_Type on x-axis
    axes[row, col_idx].set_title(f'Vegetation_Type vs {col}')
    axes[row, col_idx].set_xlabel('Vegetation_Type')  
    axes[row, col_idx].set_ylabel(col)  

# Hide any unused subplots (in case the grid has extra slots)
if num_plots % cols != 0:  # Only if there are unused subplots
    for j in range(num_plots, rows * cols):
        fig.delaxes(axes.flat[j])

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

From these violinplots, some general observations can be made.

**Altitude**: The vegetation types appear to be well-separated by altitude. Type_1 is at a higher altitude, while Type_3 is at a lower altitude, with Type_2 in between. This suggests altitude may be a good feature for distinguishing between the types.

**Slope_Orientation**: All three types show overlap in terms of slope orientation, so it doesn't seem to differentiate vegetation types strongly.

**Slope**: There is no significant distinction between the vegetation types based on slope alone, as all seem to occupy similar ranges.

**Horizontal and Vertical Distance to Water**: These variables show some degree of separation, especially for Type_3, which tends to have smaller horizontal distances to water. Type_1 and Type_2 overlap more but still show some separation.

**Shadow Index (9h, 12h, 15h)**: There’s a fair amount of overlap in the shadow indices among the vegetation types, meaning these variables may not be significant in distinguishing between them.

**Horizontal Distance to Roadways**: This feature appears to be quite distinct for Type_3, which has a smaller range and distances from roadways compared to Type_2 and Type_3.

**Horizontal Distance to Fire Points**: This variable has some separation between vegetation types, with Type_3 having much lower distances to fire points than Type_1 and Type_2, which have a wider distribution.

**Canopy Density**: All vegetation types appear to have similar canopy densities, making it difficult to differentiate between them based on this feature.

**Rainfall (Summer and Winter)**: The rainfall in both seasons seems to be very similar across vegetation types, showing little to no variation or overlap.

**Wind Exposure Level**: There is minimal distinction among the vegetation types based on wind exposure, as all three distributions seem similar.

**Key Insights**:
Altitude and Horizontal Distance to Roadways/Fire Points appear to be strong variables for separating vegetation types. Some features, like Slope Orientation, Shadow Indices, and Canopy Density, show a lot of overlap, suggesting they might not be as important in classification.
Horizontal Distance to Water also provides some separability for Type_3, which might help in classification.  

To gain more insights in the correlation between the numerical features, a heatmap is created.

In [ ]:
# Use factorize to define the categorical target variable into classes
df['Vegetation_Type'] = df.Vegetation_Type.factorize()[0]

# List of numerical columns to include in the correlation heatmap
numerical_columns = [
    'Altitude', 'Slope_Orientation', 'Slope', 
    'Horizontal_Distance_To_Water', 'Vertical_Distance_To_Water', 
    'Horizontal_Distance_To_Roadways', 'Shadow_Index_9h', 
    'Shadow_Index_12h', 'Shadow_Index_15h', 
    'Horizontal_Distance_To_Fire_Points', 'Canopy_Density', 
    'Rainfall_Summer', 'Rainfall_Winter', 'Wind_Exposure_Level',
    'Vegetation_Type'  
]

# Compute the correlation matrix
corr_matrix = df[numerical_columns].corr()

# Plot the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Heatmap of Numerical Features')
plt.show()

Some observations can be made from this Correlation Heatmap.

**Altitude**: Strong negative correlation with Vegetation_Type (-0.85), showing altitude plays a major role in distinguishing vegetation types.

**Horizontal_Distance_To_Roadway**s: Moderate negative correlation (-0.44), indicating distance from roadways helps differentiate vegetation types.

**Slope**: Positive correlation (0.37), suggesting steeper slopes are more common in certain vegetation types.

**Horizontal_Distance_To_Fire_Points**: Moderate negative correlation (-0.35), showing vegetation type is influenced by distance from fire points.

**Shadow Indices**: Little correlation with Vegetation_Type, implying minimal impact on vegetation classification.  

Some Bivariate analysis can be done on the categorical variables as well. Using stacked barcharts the distribution of the categorical variables per vegetation type can be visualized. In the graphs, the values of Vegetation_Type 0, 1 and 2 correspond to respectively Type_1, Type_2 and Type_3. As can be seen, the distribution of the *Vegetation_Type* differs highly per category of the *Soil_Type* and *Wilderness_Area*. For example, Wilderness Area 4 almost always has Vegetation Type 3. This indicates that both *Soil_Type* and *Wilderness_Area* can be relevant features in the classification problem.

In [ ]:
# List the categorical column names
categorical_columns = [
    'Soil_Type', 'Wilderness_Area']

# Set up subplots
fig, axes = plt.subplots(1, len(categorical_columns), figsize=(18, 6))
fig.suptitle('Categorical Variables vs Vegetation_Type', fontsize=16)

# Plot stacked bar charts for each categorical feature
for i, col in enumerate(categorical_columns):
    # Create a cross-tabulation of 'Vegetation_Type' against the categorical variable
    crosstab = pd.crosstab(df[col], df['Vegetation_Type'], normalize='index')
    
    # Plot the stacked bar chart
    crosstab.plot(kind='bar', stacked=True, ax=axes[i], colormap='Set2')
    axes[i].set_title(f'{col} vs Vegetation_Type')
    axes[i].set_ylabel('Proportion')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

**Summarized**:  
From this Univariate and Bivariate analysis some information can be extracted before applying any Machine Learning model. The features that are very relevant in the classification problem of the vegetation types are the altitude, slope, distances to roadways/fire points, soil type and wilderness area. On the other hand, the shadow indices seem to be of little importance in applying Machine Learning models.

### Machine Learning Methods 

**Data Preprocessing**  
Before Machine Learning models can be applied to the data, the data first needs some preprocessing. First of all, as most Machine Learning models can only handle numerical data, the categorical variables should be transformed into numerical variables. Several methods exist for this, but since our categorical data has no order, the most suitable method is OneHotEncoding. OneHotEncoding creates a new binary column for every class in the categorical columns. OneHotEncoding can be done fast using the built-in pandas function *get_dummies()*.  

In [ ]:
df = pd.get_dummies(df, columns=['Soil_Type', 'Wilderness_Area'], drop_first=True)  # drop_first to avoid dummy variable trap
df.head()

Moreover, another data preprocessing step is scaling. Scaling must happen to make sure that no features become more dominant only because they are larger in magnitude. Several methods exist for data scaling such as MinMax Scaling and Standardization. Since the features have different distributions, Standardization is used.

In [41]:
# Split into features (X) and target (y)
X = df.drop(['Id', 'Vegetation_Type'], axis=1)  # independent variables, drop Id and the target variable
y = df['Vegetation_Type']  # target variable

# Apply scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # scale the input data

#### Code multiple resampling methods

We will only give the code for LOOCV and not run it since it is very computationally expensive and a good method for small datasets. However, we have quite a big dataset.

In [42]:
# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import KFold, cross_val_score, LeaveOneOut
from sklearn.metrics import accuracy_score

# Holdout resampling
def holdout(X, y, test_size=0.3, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state) 
    
    return X_train, X_test, y_train, y_test

# k-fold cross-validation
def k_fold_cv(model, X, y, k, random_state=42):

    kf = KFold(n_splits=k, shuffle=True, random_state=random_state)
    cv_results = cross_val_score(model, X, y, cv=kf, scoring='accuracy')
    
    print(f"Cross-Validation Accuracies: {cv_results}")
    mean_accuracy = cv_results.mean()
    print(f"Mean Accuracy: {mean_accuracy:.2f}")
    
    return mean_accuracy

# If the sample is small, you can use LOOCV. Here is the code but the sample is not small so probably we won't use it
def LOOCV(model, X, y):
    # Initialize Leave-One-Out Cross-Validation
    loo = LeaveOneOut()

    # Perform LOOCV
    cv_results = cross_val_score(model, X, y, cv=loo, scoring='accuracy')

    return cv_results

def bootstrap(X, y, n_iterations=10):
    n_samples = X_scaled.shape[0]
    accuracies = []

    # Loop over the number of bootstrap iterations
    for i in range(n_iterations):
        # Create a bootstrap sample
        indices = np.random.choice(range(n_samples), size=n_samples, replace=True)
        X_train, y_train = X.iloc[indices], y.iloc[indices]
        
        # Find the out-of-bag samples (those not in the bootstrap sample)
        out_of_bag = list(set(range(n_samples)) - set(indices))
        X_test, y_test = X.iloc[out_of_bag], y.iloc[out_of_bag]
        
        # Initialize the logistic regression model
        model = LogisticRegression()
        model.fit(X_train, y_train)
        
        # Evaluate on the out-of-bag data
        if len(out_of_bag) > 0:  # Ensure there are out-of-bag samples
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracies.append(accuracy)
    
    # Return the list of accuracies from all iterations
    return accuracies


# make ROC curves
# make plot of training and test validation

#### Evaluation metrics

Define and give some theory on which evaluation metrics we will use and why we use them. R2-score, accuracy and MSE

#### Logistic Regression

We can use Linear Regression in classification problems. However, LR can result in probabilities for belonging to a certain class of below 0 and above 1. Logistic Regression makes sure that these probabilities are between 0 and 1 using a logistic function. 

### Overall Insights

- The model achieved a commendable accuracy of 84.68%, indicating a strong performance in classifying the vegetation types.
- **Type 3** demonstrated the best performance, with high precision, recall, and F1-scores, suggesting the model effectively distinguishes this class from others.
- **Type 2** exhibited lower scores compared to the other types, indicating potential challenges in correctly classifying this class, which could warrant further investigation into feature selection or model tuning.

Overall, the logistic regression model provides a good foundation for predicting vegetation types based on the features used.

#### This is without reasempling methods, so we can compare it in the end


In [ ]:
# Define target variable
y = df['Vegetation_Type']

# Define feature set (X), excluding the target column ('Vegetation_Type')
X = df.drop(['Vegetation_Type', 'Id'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Scaling (Standardize the data)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

logreg = LogisticRegression(max_iter=1000)  
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

# Step 8: Visualize the confusion matrix
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()


### Description of the Plot
- **Scatter Points**:
  - The scatter plot contains three distinct sets of points, each representing different classes. 
  - Points corresponding to Class 0 and Class 1 are clearly visible, while there is a noticeable absence of points representing Class 2 in the area where Class 1 is prevalent. 

- **Class Probability Curves**:
  - There are three probability curves plotted, corresponding to the classes:
    - **Curve 0**: This curve represents the predicted probability of Class 0. As "Altitude" increases, the probability of Class 0 decreases.
    - **Curve 1**: This curve shows the predicted probability for Class 1. It increases as "Altitude" rises, indicating a positive relationship between "Altitude" and the likelihood of belonging to Class 1.
    - **Class 2**: There is no curve for Class 2 against Class 1. This suggests that Class 2 may not have a significant relationship with "Altitude" in this dataset, as no probabilities are predicted for it.

### Observations
- **Class Separation**:
  - The curves for Class 0 and Class 1 show a clear separation based on "Altitude." As altitude increases, the probability of Class 0 decreases while the probability of Class 1 increases. This indicates that higher altitudes are associated with a higher likelihood of belonging to Class 1.
  - There is a lack of overlap between Class 2 and the other two classes, suggesting that Class 2 may be an outlier or has no representation in the altitude range observed.

- **Model Fit**:
  - The model appears to fit the data points of Classes 0 and 1 well, with the probability curves closely following the distribution of the points. However, the absence of points for Class 2 raises questions about its relevance in this context.
  
- **Uncertainty Regions**:
  - The area where the probabilities hover around 0.5 seems to be absent, indicating that the model is confident in its predictions for Classes 0 and 1 across the range of "Altitude." This lack of uncertainty might suggest a strong relationship between "Altitude" and the presence of Class 1, whereas Class 2 is not well-represented.


# TODO FIX THIS TEXT NOT SURE IF IT IS THAT TRUE


In [ ]:
X_altitude = df[['Altitude']]
feature_name = X_altitude.columns[0]  

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_altitude)

logreg = LogisticRegression(multi_class='ovr', max_iter=1000)
logreg.fit(X_scaled, y)

X_test = np.linspace(X_altitude[feature_name].min(), X_altitude[feature_name].max(), 300).reshape(-1, 1)
X_test_scaled = scaler.transform(X_test)
y_prob = logreg.predict_proba(X_test_scaled)

plt.figure(figsize=(12, 8))
plt.scatter(X_altitude[feature_name], y, color='orange', edgecolor='k', alpha=0.7, label='Data points', s=50)

for i in range(y_prob.shape[1]):
    plt.plot(X_test, y_prob[:, i], label=f'Class {i} Probability')

plt.xlabel(feature_name)
plt.ylabel('Probability')
plt.title(f'Logistic Regression Fit for {feature_name}')
plt.legend()
plt.show()


The model is robust overall, achieving an accuracy of 84.57%. I was trying to drop different features according to graphs and heatmap to try if i will have better result without all features and it didnt help me, but still it is hard to train data for type 2 because in most of the features its simillar with type 1.

In [ ]:
#Uncomment when u wanna try but if u run codes under it comment it and run again because it drops columns!
X_dropped = X.drop(['Rainfall_Summer', 'Rainfall_Winter', 'Wind_Exposure_Level', 'Canopy_Density', 'Shadow_Index_9h'], axis=1)

X_train_dropped, X_test_dropped, y_train, y_test = train_test_split(X_dropped, y, test_size=0.2, random_state=42)

scaler_dropped = StandardScaler()
X_train_dropped = scaler_dropped.fit_transform(X_train_dropped)
X_test_dropped = scaler_dropped.transform(X_test_dropped)

# Train Logistic Regression model on dropped feature set
logreg_dropped = LogisticRegression(max_iter=1000)  
logreg_dropped.fit(X_train_dropped, y_train)

# Make predictions on the test data
y_pred_dropped = logreg_dropped.predict(X_test_dropped)

# Evaluate the model
accuracy_dropped = accuracy_score(y_test, y_pred_dropped)
conf_matrix_dropped = confusion_matrix(y_test, y_pred_dropped)
class_report_dropped = classification_report(y_test, y_pred_dropped)

# Print the evaluation metrics
print(f"Accuracy (after dropping features): {accuracy_dropped}")
print("Confusion Matrix (after dropping features):")
print(conf_matrix_dropped)
print("Classification Report (after dropping features):")
print(class_report_dropped)

# Visualize the confusion matrix
sns.heatmap(conf_matrix_dropped, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix (after dropping features)')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()


## LDA 
The Linear Discriminant Analysis (LDA) model achieved an overall accuracy of approximately 83.51% after dropping certain features. While the model performed well in predicting Class 0 and Class 2, there is room for improvement in Class 1, where both precision and recall are lower. Overall, the model demonstrates good performance. I used it with dropped features because in this method i had better accuracy when i dropped them.

But its also not optimal for our dataset because it should have better performance when features are `normally` distributed.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assuming df is your DataFrame and y is the target variable
y = df['Vegetation_Type']
X = df.drop(['Vegetation_Type', 'Id'], axis=1)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Scaling
scaler_dropped = StandardScaler()
X_train_dropped = scaler_dropped.fit_transform(X_train)
X_test_dropped = scaler_dropped.transform(X_test)

# Initialize and fit the LDA model
lda_dropped = LinearDiscriminantAnalysis()
lda_dropped.fit(X_train_dropped, y_train)

# Make predictions on the test set
y_pred_dropped = lda_dropped.predict(X_test_dropped)

# Evaluate the model
accuracy_dropped = accuracy_score(y_test, y_pred_dropped)
conf_matrix_dropped = confusion_matrix(y_test, y_pred_dropped)
class_report_dropped = classification_report(y_test, y_pred_dropped)

# Print evaluation metrics
print(f"Accuracy (after dropping features): {accuracy_dropped}")
print("Confusion Matrix (after dropping features):")
print(conf_matrix_dropped)
print("Classification Report (after dropping features):")
print(class_report_dropped)

# Visualize the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_dropped, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix (after dropping features)')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()


# Conclusion

In this analysis, we utilized Linear Discriminant Analysis (LDA) to visualize the decision boundaries of our model based on selected features from the dataset. Specifically, we focused on the 14th column and the last column of the dataset.

The resulting plot illustrated how the LDA model distinguishes between different classes based on the values of the chosen features. The color gradient in the background represents the predicted class regions, while the scatter plots indicate the actual training and test data points. 

- **Training Data**: Shown with circular markers, they indicate the distribution of the classes within the training dataset.
- **Test Data**: Represented by cross markers, they provide insight into how well the model predicts unseen data.

The decision boundaries reflect the relationships between the selected features and their influence on class separation. Overall, the visualization highlights the effectiveness of LDA in classifying the data based on the chosen features, as well as the potential areas of overlap where class distinctions may be less clear.

## CHECK WITH TEACHER IF THIS GRAPH IS EVEN OK?

In [ ]:
X_vis = X_train_dropped[:, [13, -1]]

feature_names = X.columns[[13, -1]]

print("Selected features for visualization (14th column and last column):")
print(feature_names.tolist())
print(X_vis)

lda_dropped_vis = LinearDiscriminantAnalysis()
lda_dropped_vis.fit(X_vis, y_train)

x_min, x_max = X_vis[:, 0].min() - 1, X_vis[:, 0].max() + 1
y_min, y_max = X_vis[:, 1].min() - 1, X_vis[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                     np.arange(y_min, y_max, 0.1))

Z = lda_dropped_vis.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.figure(figsize=(12, 8))
contour = plt.contourf(xx, yy, Z, alpha=0.5, cmap='Spectral')

cbar = plt.colorbar(contour)
cbar.set_label('Predicted Class', rotation=270, labelpad=15)

plt.scatter(X_vis[:, 0], X_vis[:, 1], c=y_train, edgecolor='k', s=120, cmap='Spectral', marker='o', label='Training Data', alpha=0.9)

X_test_vis = X_test_dropped[:, [13, -1]]
plt.scatter(X_test_vis[:, 0], X_test_vis[:, 1], c=y_test, edgecolor='k', marker='x', s=120, label='Test Data', alpha=0.2)

plt.title('LDA Decision Boundaries')
plt.xlabel(feature_names[0])
plt.ylabel(feature_names[1])
plt.legend()
plt.grid(True)
plt.show()


# QDA
QDA is a versatile classification method that shines in scenarios where non-linear decision boundaries, class-specific variances, and multi-class situations are present. Its ability to model complex relationships and robustness in handling class imbalances makes it a valuable tool in the data scientist's toolkit.

In our dataset we can see its not that good for using.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assuming df is your DataFrame and y is the target variable
y = df['Vegetation_Type']
X = df.drop(['Vegetation_Type', 'Id'], axis=1)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Scaling
scaler_dropped = StandardScaler()
X_train_dropped = scaler_dropped.fit_transform(X_train)
X_test_dropped = scaler_dropped.transform(X_test)

# Initialize and fit the LDA model
qda_dropped = QuadraticDiscriminantAnalysis()
qda_dropped.fit(X_train_dropped, y_train)

# Make predictions on the test set
y_pred_dropped = qda_dropped.predict(X_test_dropped)

# Evaluate the model
accuracy_dropped = accuracy_score(y_test, y_pred_dropped)
conf_matrix_dropped = confusion_matrix(y_test, y_pred_dropped)
class_report_dropped = classification_report(y_test, y_pred_dropped)

# Print evaluation metrics
print(f"Accuracy (after dropping features): {accuracy_dropped}")
print("Confusion Matrix (after dropping features):")
print(conf_matrix_dropped)
print("Classification Report (after dropping features):")
print(class_report_dropped)

# Visualize the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_dropped, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix (after dropping features)')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()



In [ ]:
# Build the logistic regression model
log_reg = LogisticRegression()

X_train, X_test, y_train, y_test = holdout(X_scaled, y, test_size=0.3)
k_fold_cv(log_reg, X_scaled, y, 5)
#LOOCV(log_reg, X_scaled, y) --> takes very long since dataset is quite big

In [ ]:
# Train the model
log_reg.fit(X_train, y_train)

# Make predictions
y_pred = log_reg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print results
print(f'Accuracy: {accuracy * 100:.2f}%')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

In [51]:
# make a plot

#### Linear Discriminant Analysis

explain when to use LDA and if this applies

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

# Build the linear discriminant analysis model
lda = LinearDiscriminantAnalysis()

# Train the model
lda.fit(X_train, y_train)

# Make predictions
y_pred_lda = lda.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred_lda)
conf_matrix = confusion_matrix(y_test, y_pred_lda)
class_report = classification_report(y_test, y_pred_lda)

# Print results
print(f'Accuracy: {accuracy * 100:.2f}%')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

In [53]:
# make plots

#### Quadrant Discriminant Analysis

explain when to use QDA and evaluate if this is necessary

In [ ]:
# Build the linear discriminant analysis model
qda = QuadraticDiscriminantAnalysis()

# Train the model
qda.fit(X_train, y_train)

# Make predictions
y_pred_qda = qda.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred_qda)
conf_matrix = confusion_matrix(y_test, y_pred_qda)
class_report = classification_report(y_test, y_pred_qda)

# Print results
print(f'Accuracy: {accuracy * 100:.2f}%')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

### LDA vs QDA --> check covariance matrices of classes

If the covariance matrices are similar across classes, LDA should work well; if they are different, QDA is likely the better choice.

In [ ]:
classes = df['Vegetation_Type'].unique()
df_cov = df.drop(['Id', 'Soil_Type', 'Wilderness_Area'], axis=1)  # independent variables
covariances = {}

for class_value in classes:
    class_data = df_cov[df_cov['Vegetation_Type'] == class_value].drop('Vegetation_Type', axis=1)  # Drop target column
    covariances[class_value] = np.cov(class_data, rowvar=False)

# Output covariance matrices for each class
for class_value, cov_matrix in covariances.items():
    print(f"Covariance matrix for class {class_value}:\n", cov_matrix, "\n")

#### Feature selection using regularization methods

Removing features can improve understandability and performance of the model. Feature selection can be done automatically Ridge, Lasso or Elastic Net Regression. When coefficients of variables become 0 or increasingly small, these features can be removed from the classification model to improve model interpretability and performance. 

In [ ]:
# Define the Ridge regression model
ridge = Ridge()

# Set up a grid of alpha values to try
alpha_values = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}  # You can adjust this range

# Set up GridSearchCV to tune alpha
grid_search = GridSearchCV(ridge, param_grid=alpha_values, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the model to the training data
grid_search.fit(X_train, y_train)

# Get the best model (best alpha)
best_ridge = grid_search.best_estimator_
print(f"Best alpha value: {grid_search.best_params_['alpha']}")

# Predict on the test set using the best model
y_pred = best_ridge.predict(X_test)

# Evaluate the model using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error with best alpha: {mse}")

# Print the coefficients along with the corresponding feature names
coefficients = best_ridge.coef_
feature_names = X.columns

# Combine the feature names and their corresponding coefficients
coeff_df = pd.DataFrame({'Feature': feature_names, 'Ridge Coefficient': coefficients})

# Display the coefficients
print("\nRidge Coefficients with best alpha:")
print(coeff_df)

In [ ]:
# Define the Lasso regression model
lasso = Lasso()

# Set up a grid of alpha values to try
alpha_values = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}  # You can adjust this range

# Set up GridSearchCV to tune alpha
grid_search = GridSearchCV(lasso, param_grid=alpha_values, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the model to the training data
grid_search.fit(X_train, y_train)

# Get the best model (best alpha)
best_lasso = grid_search.best_estimator_
print(f"Best alpha value: {grid_search.best_params_['alpha']}")

# Predict on the test set using the best model
y_pred = best_lasso.predict(X_test)

# Evaluate the model using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error with best alpha: {mse}")

# Print the coefficients along with the corresponding feature names
coefficients = best_lasso.coef_
feature_names = X.columns

# Combine the feature names and their corresponding coefficients
coeff_df = pd.DataFrame({'Feature': feature_names, 'Lasso Coefficient': coefficients})

# Display the coefficients
print("\nLasso Coefficients with best alpha:")
print(coeff_df)

In [ ]:
# try ridge regression with multiple alphas, make a plot of the coefficients and the L2 norm
# make plot of bias-variance trade-off and use CV for selecting alpha
lambdas = 10**np.linspace(10, -2, 100)*0.5

ridge = Ridge()
coefs = []

for l in lambdas:
    ridge.set_params(alpha = l)
    ridge.fit(X_scaled, y)
    coefs.append(ridge.coef_)

np.shape(coefs)

In [ ]:
# Plot the coefficients as they change over values of lambda
ax = plt.gca()
ax.plot(lambdas, coefs)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('lambda')
plt.ylabel('weights')

In [ ]:
ridge2 = Ridge(alpha = 0.0000001)
ridge2.fit(X_train, y_train)
pred2 = ridge2.predict(X_test)
print(pd.Series(ridge2.coef_))
print(mean_squared_error(y_test, pred2))

In [ ]:
ridgecv = RidgeCV(alphas = lambdas, scoring = 'neg_mean_squared_error')
ridgecv.fit(X_train, y_train)
ridgecv.alpha_

In [ ]:
ridge4 = Ridge(alpha = ridgecv.alpha_)
ridge4.fit(X_train, y_train)
mean_squared_error(y_test, ridge4.predict(X_test))

In [ ]:
# make a bias-variance trade-off plot
bias_sq = []
variance = []
mse = []

n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

for l in lambdas:
    y_preds = np.zeros((X_scaled.shape[0], n_splits))

    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X_scaled)):
        X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        ridge.set_params(alpha = l)
        ridge.fit(X_train, y_train)

        y_preds[test_idx, fold_idx] = ridge.predict(X_test).flatten()

    avg_preds = np.mean(y_preds, axis=1)
    bias_sq.append(np.mean((avg_preds - y.to_numpy().flatten())**2))
    variance.append(np.mean(np.var(y_preds, axis=1)))
    mse.append(bias_sq[-1] + variance[-1])

plt.figure(figsize=(10, 6))
plt.plot(lambdas, bias_sq, label="Bias^2", color="blue", linewidth=2)
plt.plot(lambdas, variance, label="Variance", color="green", linewidth=2)
plt.plot(lambdas, mse, label="Total MSE", color="red", linestyle='--', linewidth=2)
plt.xscale('log')  # Use log scale for lambda
plt.xlabel("Lambda (alpha)")
plt.ylabel("Error")
plt.title("Bias-Variance Trade-off in Ridge Regression (Cross-Validation)")
plt.legend()
plt.show()


In [ ]:
lasso = Lasso(max_iter = 10000)
coefs_lasso = []

for l in lambdas:
    lasso.set_params(alpha = l)
    lasso.fit(X_scaled, y)
    coefs_lasso.append(lasso.coef_)

# Plot the coefficients as they change over values of lambda
ax = plt.gca()
ax.plot(lambdas, coefs_lasso)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('lambda')
plt.ylabel('weights')

In [ ]:
lassocv = LassoCV(alphas = None, cv = 10, max_iter = 100000)
lassocv.fit(X_train, y_train)

lasso.set_params(alpha=lassocv.alpha_)
lasso.fit(X_train, y_train)
mean_squared_error(y_test, lasso.predict(X_test))

In [ ]:
r2_score(y_test,  lasso.predict(X_test))

In [ ]:
pd.Series(lasso.coef_, index=X.columns) 

# Feature Selection

There are two major problems related to training models: **overfitting** and **underfitting**:

- Overfitting: The model performs well on the training set but not so well on unseen (test) data.
- Underfitting: Neither performs well on the train set nor on the test set.

**Regularization** is implemented to avoid overfitting of the data, especially when there is a large variance between train and test set
performances. There are different methods of reducing the model complexity and preventing overfitting in linear models which are *Ridge* and *Lasso*
*Regression Models*.

## Preparing Data for Ridge Regression

Before applying Ridge Regression, we need to properly prepare the dataset. This involves several steps, such as:

1. **Feature Selection/Engineering**: Ensure relevant features are selected and encoded.
2. **Splitting the Data**: Divide the dataset into training and test sets.
3. **Feature Scaling**: Apply scaling to ensure features are on a similar scale, as Ridge Regression is sensitive to feature magnitudes.

Below is an example of how to prepare data for Ridge Regression.

In [56]:
# Import all libraries 
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet 
from sklearn.metrics import mean_squared_error , root_mean_squared_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler 
import numpy as np  
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

def prepare_dataset(df, target, categorical_features, numerical_features, numberical_target_var = True):

    df.isnull().sum()*100/df.shape[0]
    df.head()
    # Encode the target variable if it's categorical
    if numberical_target_var == False:
        le = LabelEncoder()
        y = le.fit_transform(df[target])
    
    # Convert the categorical columns into the boolean type, remove the Id col
    dummies = pd.get_dummies(df[categorical_features])

    # Drop the categorical features from the X
    X_ = df.drop(categorical_features, axis = 1) 
    # Drop the target variable
    X_ = X_.drop(target, axis = 1) 
    X_ = X_.drop(["Id"], axis=1) # TODO remove

    # Convert all numerical values to the float type
    X = X_.apply(lambda col: col.astype('float64') if col.dtype in ['int64', 'float32', 'float64'] else col)
    # Merge the dummies (categorical var w numerical variables)
    X = pd.concat([X, dummies], axis = 1) 

    return X, y


In [ ]:
# Display the head of the dataset before any modification
# df = df.drop("Vegetation_Type_Encoded", axis=1)
df = pd.concat([df_1, df_2, df_3], ignore_index=True)
df.head(5)
df.info()

In [ ]:
# Drop the column with the independent variable (Salary), and columns for which we created dummy variables 
y = df.Vegetation_Type
le = LabelEncoder()
y = le.fit_transform(y)

numerical_features = ["Altitude", "Slope_Orientation", "Slope", "Horizontal_Distance_To_Water"
    ,"Vertical_Distance_To_Water", "Horizontal_Distance_To_Roadways", "Shadow_Index_9h",
    "Shadow_Index_12h", "Shadow_Index_15h", "Horizontal_Distance_To_Fire_Points", 
    "Canopy_Density", "Rainfall_Summer", "Rainfall_Winter", "Wind_Exposure_Level"]
categorical_features = ["Wilderness_Area", "Soil_Type"]
target_variable = "Vegetation_Type"

print(df.columns)

# Get x,y data
X, y = prepare_dataset(df, target_variable, categorical_features, numerical_features, False)
# Show info about X data
X.info()


In [66]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.5, random_state=42)
sc=StandardScaler() 
X_train=sc.fit_transform(X_train) 
X_test=sc.transform(X_test)

#from sklearn.preprocessing import minmax_scale 
# minmax_scale = MinMaxScaler()
# X_train=minmax_scale(X_train, axis=0) 
# X_test=sc.transform(X_test) 

In [67]:
# Generate an NumPy array of alphas from small to large 
# number which are logarithmically spaced, meaning they decrease exponentially

alphas = 10**np.linspace(10,-2,100)*0.5 

# Uncomment the alphas if you want to see the array
# alphas 

## Ridge Resgression
Is a variation of linear regression, specifically designed to address multicollinearity in the dataset. In linear regression, the goal is to find the best-fitting hyperplane that minimizes the sum of squared differences between the
observed and predicted values, but when there is high correlation between variables, LR model may be moderately or highly correlated with another.
Multicollinearity exists when 2 or more predictors in regression model are correlated with another one. 

**Ridge Regression** use *L2 penalty*, that penalize the large coefficients to prevent overfitting.

The tuning parameter λ serves to control the relative impact, when λ = 0 than penalty has no effect. As λ grows to infitite the penalty grows which lead to shrinking coeffiecients to zero. Cross-Validation is used for selecting a good value for λ as it's very important.

In [ ]:
"""
In this cell we perform Ridge Regression for multiple alpha values that were 
generated in the cell before and calculate the coefficients, MSE, Bias and Variance for each model.
Data are then used for plots.
"""

ridge = Ridge() 
coefs = [] 

mse = []
squared_bias = []
variance = []
 
for a in alphas: 
    ridge.set_params(alpha = a) 
    ridge.fit(X, y) 
    coefs.append(ridge.coef_) 
    y_pred = ridge.predict(X_test)

     # Calculate MSE
    mse.append(np.mean((y_pred - y_test) ** 2))
    
    # Calculate bias and variance
    bias = np.mean(y_pred - y_test)
    squared_bias.append(bias ** 2)
    variance.append(np.var(y_pred))
     
np.shape(coefs)


### Plot description
#### Plot axis
This snippet of code visualizes how the coefficients (weights) of a Ridge Regression model change across different values of the regularization parameter, alpha. 
- X-axis: 
    - This is the regularization parameter for Ridge Regression.
    - Smaller alpha values mean less regularization 
    - Larger alpha values mean more regularization, which shrinks the coefficients towards zero.
- Y-axis
    - Each line corresponds to the coefficient value (weight) of a feature in the model.

### Plot behaviour
- Small alpha values
    - Coefficients are large (the model tries to fit the training data closely).
- As alpha increases
    - Coefficients shrink towards zero, indicating regularization
    - If coefficient shrink to exactly 0 that mean feature is ignored in model
- Very high alpha
    - All coefficients tend towards zero, and the model behaves like a simple mean-based model 

### Key sights
- Lower alpha indicate more complex model but potentially overfitting
- Higher alpha stabilize coefficients or shrink to zero, indicating simpler/more stable model but may underfit data
- If coefficients drop to zero early, that suggest that feature **may not cotribute to much** in model

In [ ]:
ax = plt.gca() 
ax.plot(alphas, coefs) 
ax.set_xscale('log') 
plt.axis('tight') 
plt.xlabel('alpha') 
plt.ylabel('weights')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(alphas, squared_bias, color='black', label='Squared Bias', linewidth=2)
plt.plot(alphas, variance, color='green', label='Variance', linewidth=2)
plt.plot(alphas, mse, color='purple', label='Test MSE', linewidth=2)
plt.xscale('log')
plt.axhline(y=min(mse), color='red', linestyle='--', label='Minimum MSE')
plt.scatter(alphas[np.argmin(mse)], min(mse), color='purple', marker='x', s=100, label='Minimum MSE Point')
plt.title('Ridge Regression: Bias, Variance, and MSE vs. Regularization Parameter (λ)')
plt.xlabel('Regularization Parameter (λ)')
plt.ylabel('Error')
plt.legend()
plt.grid()
plt.show()

Instead of arbitrarily choosing alpha's, we use cross-validation to choose the tuning parameter alpha. We can do this using the cross-
validated ridge regression function, RidgeCV(). By default, the function performs generalized cross-validation (an efficient form of
LOOCV), though this can be changed using the argument cv. Once the RidgeCV finished, we will display the alpha value and use it in Ridge function to get the MSE, Score and see the coefficients values.

In [ ]:
from sklearn.linear_model import RidgeCV, Ridge
from sklearn.metrics import r2_score
 
# Use the Cross Validation to calculate the best alpha
ridgecv = RidgeCV(alphas = alphas, scoring = 'neg_mean_squared_error') 
ridgecv.fit(X_train, y_train)
ridgecv.alpha_

In [ ]:
# Assuming X_train and X_test are DataFrames
ridge4 = Ridge(alpha=ridgecv.alpha_) 
ridge4.fit(X_train, y_train)
y_pred = ridge4.predict(X_test)

# Calculate and print MSE and R² Score
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.5f}")
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")
coefficients = ridge4.coef_

coef_series = pd.Series(ridge4.coef_, index=X.columns)
sorted_coef = coef_series.reindex(coef_series.abs().sort_values(ascending=False).index)

print("Coefficients sorted by absolute value:")
print(sorted_coef)

Lasso (Least Absolute Shrinkage and Selection Operator) is a type of linear regression that uses L1 regularization to prevent overfitting by penalizing the absolute size of regression coefficients.
- It adds a penalty equivalent to the sum of the absolute values of coefficients to the loss function
- Lasso shrinks some coefficients to exactly zero, effectively selecting a subset of features. This makes it a good choice when dealing with high-dimensional data or when some features are irrelevant
- By shrinking coefficients, Lasso reduces variance at the cost of introducing some bias, helping to improve generalization

In [ ]:
from sklearn.linear_model import Lasso, LassoCV 
from sklearn.preprocessing import scale  
 
lasso = Lasso(max_iter = 10000) 
coefs = [] 


mse = []
squared_bias = []
variance = []
 
for a in alphas: 
    lasso.set_params(alpha=a) 
    lasso.fit(scale(X_train), y_train) 
    coefs.append(lasso.coef_) 

    y_pred = lasso.predict(X_test)

     # Calculate MSE
    mse.append(np.mean((y_pred - y_test) ** 2))
    
    # Calculate bias and variance
    bias = np.mean(y_pred - y_test)
    squared_bias.append(bias ** 2)
    variance.append(np.var(y_pred))
     
np.shape(coefs)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(alphas, squared_bias, color='black', label='Squared Bias', linewidth=2)
plt.plot(alphas, variance, color='green', label='Variance', linewidth=2)
plt.plot(alphas, mse, color='purple', label='Test MSE', linewidth=2)
plt.xscale('log')
plt.axhline(y=min(mse), color='red', linestyle='--', label='Minimum MSE')
plt.scatter(alphas[np.argmin(mse)], min(mse), color='purple', marker='x', s=100, label='Minimum MSE Point')
plt.title('Ridge Regression: Bias, Variance, and MSE vs. Regularization Parameter (λ)')
plt.xlabel('Regularization Parameter (λ)')
plt.ylabel('Error')
plt.legend()
plt.grid()
plt.show()

In this section we are using the Cross Validation for selecting the best alpha value as it's critical part of the Regression. Once we have the alpha value we Train with Lasso using best alpha, predictions are made. After that we calculate the metrics as MSE and Score, and show the relevant results from model to see coefficients of different columns.

In [ ]:
from sklearn.linear_model import Lasso, LassoCV
import numpy as np
import pandas as pd

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=42).fit(X_train, y_train)

best_alpha = lasso_cv.alpha_
print(f"Best Alpha: {best_alpha}")

lasso = Lasso(alpha=best_alpha)
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_test)

mse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")

coefficients = pd.Series(lasso.coef_, index=X.columns)
print("\nCoefficients (Feature Name: Value):")
print(coefficients)

features_to_drop = coefficients[coefficients == 0]

print("\nFeatures to Drop (Feature Name: Coefficient):")
for feature, coef in features_to_drop.items():
    print(f"{feature}: {coef}")

